In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)
from scipy.interpolate import griddata
import netCDF4 as nc4
from netCDF4 import Dataset
from datetime import datetime, timedelta

In [2]:
dataset = Dataset('sic_day_MPI-ESM-MR_historical_r1i1p1_20000101-20001231.nc')
print dataset.file_format

NETCDF3_CLASSIC


In [3]:
print dataset.variables.keys()
time = dataset.variables['time']
lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]
sic =  dataset.variables['sic'][:]

[u'time', u'time_bnds', u'j', u'i', u'lat', u'lon', u'lat_vertices', u'lon_vertices', u'sic']


In [15]:
dtime = sorted(list(nc4.num2date(time[:],time.units)))


In [5]:
f = open('sic_echam6_t.txt','w')
for i in range(len(dtime)):
    val = str(dtime[i].strftime(' %Y%m%d %H%M%S')) +'\n' 
    f.write(val)   
    df = pd.DataFrame({'lon1':lon.ravel(),'lat1':lat.ravel(),'sic1':sic[i,:,:].ravel()})
    x = (df['lon1'])
    y = (df['lat1'])
    z = (df['sic1'])
    def grid(x_min, x_max, y_min, y_max, x_step, y_step):
        xi = np.arange(x_min, x_max, x_step)
        yi = np.arange(y_min, y_max, y_step)
        xx, yy = np.meshgrid(xi, yi)
        return xi, yi, xx, yy
    xi, yi, xx, yy = grid(0, 360, -90, 90,1.85, 1.85)
    griddata((x,y),z,(xx,yy))
    zi = griddata((df['lon1'].values, df['lat1'].values),df['sic1'].values,(xx, yy),method='linear')
    zi1 = zi/100
    np.savetxt(f,zi1,fmt='%8.2f', delimiter="")
    i +=1
f.close()                            